In [35]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import os
from glob import glob
import numpy as np

from PIL import Image

In [2]:
seed =1

lr = 0.001#learning rate
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,20,5,1);#input,output,filter size,stride
    self.conv2 = nn.Conv2d(20,50,5,1);
    self.fc1 = nn.Linear(4*4*50,500);
    self.fc2 = nn.Linear(500,10)
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2)
    x = x.view(-1,4*4*50)#1자로 피기
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim = 1)

In [4]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
!unzip /content/gdrive/"My Drive"/fastCampus/mnist_png.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: mnist_png/training/9/18211.png  
  inflating: mnist_png/training/9/18228.png  
  inflating: mnist_png/training/9/18241.png  
  inflating: mnist_png/training/9/18251.png  
  inflating: mnist_png/training/9/1826.png  
  inflating: mnist_png/training/9/18277.png  
  inflating: mnist_png/training/9/18296.png  
  inflating: mnist_png/training/9/183.png  
  inflating: mnist_png/training/9/18350.png  
  inflating: mnist_png/training/9/18355.png  
  inflating: mnist_png/training/9/18374.png  
  inflating: mnist_png/training/9/18377.png  
  inflating: mnist_png/training/9/18382.png  
  inflating: mnist_png/training/9/18387.png  
  inflating: mnist_png/training/9/18401.png  
  inflating: mnist_png/training/9/18405.png  
  inflating: mnist_png/training/9/18415.png  
  inflating: mnist_png/training/9/18437.png  
  inflating: mnist_png/training/9/18441.png  
  inflating: mnist_png/training/9/18448.png  
  inflating: mnist_png/training/9/18458.png  
 

In [6]:
train_dir = 'mnist_png/training/'
test_dir = 'mnist_png/testing/'

In [7]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [8]:
!nvidia-smi

Tue Jul 28 13:51:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
train_dataset = datasets.ImageFolder(root = train_dir,
                                     transform=transforms.Compose([
                                                                   transforms.ToTensor(),#0~1사이의 값으로 만들어줌
                                                                   transforms.Normalize((0.1307,),(0.3081,))#시발 왜 이렇게하징?ㅋㅋ
                                     ]))
#flow_from_directory 같은거임
#augumentation을 같이 해준다.
#grayscale이 안된다.
test_dataset = datasets.ImageFolder(root = test_dir,
                                     transform=transforms.Compose([
                                                                   transforms.ToTensor(),#0~1사이의 값으로 만들어줌
                                                                   transforms.Normalize((0.1307,),(0.3081,))#시발 왜 이렇게하징?ㅋㅋ
                                     ]))

In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

#iterator로 만들어준다.

#여기부턴 customize dataset


In [16]:
!unzip /content/gdrive/"My Drive"/fastCampus/cifar.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: cifar/train/5501_deer.png  
  inflating: cifar/train/5502_cat.png  
  inflating: cifar/train/5503_dog.png  
  inflating: cifar/train/5504_dog.png  
  inflating: cifar/train/5505_dog.png  
  inflating: cifar/train/5506_bird.png  
  inflating: cifar/train/5507_cat.png  
  inflating: cifar/train/5508_frog.png  
  inflating: cifar/train/5509_dog.png  
  inflating: cifar/train/550_cat.png  
  inflating: cifar/train/5510_dog.png  
  inflating: cifar/train/5511_truck.png  
  inflating: cifar/train/5512_automobile.png  
  inflating: cifar/train/5513_frog.png  
  inflating: cifar/train/5514_deer.png  
  inflating: cifar/train/5515_automobile.png  
  inflating: cifar/train/5516_airplane.png  
  inflating: cifar/train/5517_dog.png  
  inflating: cifar/train/5518_horse.png  
  inflating: cifar/train/5519_truck.png  
  inflating: cifar/train/551_horse.png  
  inflating: cifar/train/5520_cat.png  
  inflating: cifar/train/5521_airplane.png  
  inflati

In [17]:

train_paths = glob('cifar/train/*.png')
test_paths = glob('cifar/test/*.png')
len(train_paths), len(test_paths)

(50000, 10000)

In [21]:
path = train_paths[0]
path

'cifar/train/39042_dog.png'

In [20]:
lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
lbl_name

'dog'

In [24]:

label_names = [get_label(path) for path in train_paths]
classes = np.unique(label_names)

In [31]:
def get_label(path):
  lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
  label = np.argmax(classes == lbl_name)
  return label

In [27]:
np.argmax(classes == get_label(path))
#label을 얻었다.

5

In [30]:
img_pil = Image.open(path)
img = np.array(img_pil)
img.shape

(32, 32, 3)

In [57]:
#양식으로 있다.
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        #read image
        path = self.data_paths[idx]#index에 따라 경로를 받아준다.
        
        image = Image.open(path)

        label = get_label(path)
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [58]:
kwargs = {'num_workers':1,'pin_memory':True} if use_cuda else{}



In [59]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths,
            transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean = [0.406],
                                    std = [0.225]
                                )
            ])
            ),
    batch_size = batch_size,
    shuffle = True,
    **kwargs
)

In [60]:
test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
            transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean = [0.406],
                                    std = [0.225]
                                )
            ])
            ),
    batch_size = batch_size,
    shuffle = False,
    **kwargs
)

In [63]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,20,5,1);#input,output,filter size,stride
    self.conv2 = nn.Conv2d(20,50,5,1);
    self.fc1 = nn.Linear(5*5*50,500);
    self.fc2 = nn.Linear(500,10)
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2)
    x = x.view(-1,5*5*50)#1자로 피기
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim = 1)

In [64]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.300774
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.289062
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.283774
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.247731
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.281374
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.144951
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.183564
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.104105

Test set: Average loss: 2.0656, Accuracy: 2782/10000 (28%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.102824
Train Epoch: 2 [6400/50000 (13%)]	Loss: 2.072438
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.955644
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.971324
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.881922
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.955828
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.987089
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.992772
